In [1]:
SEED = 42
PYTORCH_PRETRAINED_BERT_CACHE = "~/.pytorch_pretrained_bert"

In [2]:
import json
import os
import sys
import gc
import torch
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import itertools
import logging
import datetime

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from pytorch_pretrained_bert.modeling import BertForNextSentencePrediction
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear, SCHEDULES

from fastprogress import master_bar, progress_bar
from tensorboardX import SummaryWriter

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
import argparse
from scipy.misc import imread, imresize
from PIL import Image

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
# old_stdout = sys.stdout
# log_file = open("output.log", "w")
# sys.stdout = log_file

In [4]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == torch.device("cuda"):
    torch.cuda.manual_seed_all(SEED)

In [5]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger("bert")
writer = SummaryWriter('runs/{}'.format(str(datetime.datetime.now())))

In [6]:
def __add_diversity(
        beam_seqs,
        logprobs,
        time,
        group_number,
        penalty_lambda,
        group_size):
    """ Adds a diversity penalty to a group of beams
    beam_seqs       : array containing beam sequences staggered across time
    logprobs        : log probabilities for the beam sequences
    time            : Current time unit (not adjusted for the current group
    group_number    : the current group number
    penalty_lambda  : diversity penalty
    group_size      : num_beams/num_groups
    """
    local_time = time - group_number # current time for the group
    aug_logprobs = logprobs.clone().to(device)
    for previous_choice in range(group_number):
        previous_decisions = beam_seqs[previous_choice][local_time]
        for beam in range(group_size):
            for previous_labels in range(group_size):
                aug_logprobs[beam][previous_decisions[previous_labels]] -= (
                    penalty_lambda
                ) # penalize previously chosen words
    return aug_logprobs


In [7]:
def __beam_step(
        logprobs,
        aug_logprobs,
        beam_size,
        beam_seq,
        beam_seq_logprobs,
        beam_logprobs_sum,
        rnn_state,
        time):
    """ Runs one step of beam search
    logprobs         : log probabilities for beam_seqs
    aug_logprobs     : log probabilities after penalty
    beam_size        : Beam Size
    beam_seq         : Tensor containing the beams
    beam_seq_logprobs: log-probabilities of each beam sequence
    beam_logprobs_sum: joint log probability of each beam
    time             : time step
    """
#     print("aug log probs size: ", aug_logprobs.shape)
    ys, indices = torch.sort(aug_logprobs, 1, True)
    candidates = []
    columns = min(beam_size, len(ys[0]))
    rows = beam_size
#     print(beam_size)
    if time == 1:
        rows = 1

    for column in range(columns):
        for row in range(rows):
            local_logprob = ys[row,column]
            candidate_logprob = beam_logprobs_sum[row] + local_logprob
            local_unaugmented_logprob = logprobs[row,indices[row][column]]
            candidates.append(
                {
                    "column":indices[row][column],
                    "row":row,
                    "prob":candidate_logprob,
                    "local_logprob":local_unaugmented_logprob
                }
            )
    candidates.sort(key=lambda x: x['prob'])
#     print(rnn_state)
    new_state = [x.clone().to(device) for x in rnn_state]
#     print("new state: ", len(new_state))
#     print("rnn_state: ", len(rnn_state), type(rnn_state[0]), rnn_state[0].shape)
#     print("candidates: ", len(candidates))

    if time > 1:
        beam_seq_prev = beam_seq[0:time,:].clone()
        beam_seq_logprobs_prev = beam_seq_logprobs[0:time, :].clone()

    for v_index in range(beam_size):
        candidate = candidates[v_index]
        candidate['kept'] = True
        if time > 1:
            beam_seq[0:time, v_index] = (
                beam_seq_prev[:,candidate['row']]
            )
            beam_seq_logprobs[0:time, v_index] = (
                beam_seq_logprobs_prev[:,candidate['row']]
            )
        for state_index in range(len(new_state)):
#             print("\n\n\n",v_index)
            new_state[state_index][v_index] = (
                rnn_state[state_index][candidate['row']]
            )

        beam_seq[time,v_index] = candidate['column']
        beam_seq_logprobs[time, v_index] = candidate['local_logprob']
        beam_logprobs_sum[v_index] = candidate['prob']
    state = new_state
    return (
        beam_seq,
        beam_seq_logprobs,
        beam_logprobs_sum,
        state,
        candidates
    )


In [8]:
def diverse_beam_search(
        rnn_state,
        init_logprobs,
        num_beams,
        num_groups,
        penalty_lambda,
        seq_length,
        end_token,
        gen_logprobs,
        init_token):
    """ Performs Diverse Beam Search
    seq_length: maximum length of sequence
    rnn_state: states of the RNNs
    logprobs: log-probabilities of the beams
    num_beams: number of beams
    num_groups: number of groups
    penalty_lambda: value of diversity penalty
    end_token: end-token of the vocabulary
    gen_logprobs: function that returns
    the states and logprobs from the RNN output
    """
    beam_ratio = int(num_beams / num_groups)
    states = []
    beam_seqs = []
    beam_seq_logprobs = []
    beam_logprobs_sums = []
    done_beams = []
    logprobs_list=[]
    state=[]
    # Initialization
#     print("BEAM RATIO ", beam_ratio, "\n", type(beam_ratio))

    state = [None]*beam_ratio
#     print(state)

    for group_num in range(num_groups):
        beam_seqs.append(torch.zeros(
            seq_length, beam_ratio, dtype=torch.long
        ).to(device))

        beam_seq_logprobs.append(torch.zeros(
            seq_length, beam_ratio
        ).to(device))

        beam_logprobs_sums.append(torch.zeros(
            beam_ratio
        ).to(device))

        for k in range(beam_ratio):
            done_beams.append(None)
        logprobs_list.append(torch.zeros(
            beam_ratio, init_logprobs.size()[1]
        ).to(device))

        logprobs_list[group_num] = (
            init_logprobs.clone().to(device)
        )

        for sub_beam_ix in range(beam_ratio):

            beam_seqs[group_num][sub_beam_ix][0] = init_token
            state[sub_beam_ix] = [st.clone().to(device) for st in rnn_state]

#             print(state)

        states.append(
            [item.clone().to(device) for item in rnn_state]
        )
    # End initialization

    for time in range(seq_length + num_groups):
        for group_ix in range(num_groups):
            if time >= group_ix and time < seq_length + group_ix:
                logprobs = logprobs_list[group_ix]

                # Suppress <UNK> tokens in the decoding
                logprobs[:,-2] -= 1000
#                 print("logprobs shape: ", logprobs.shape)
                aug_logprobs = __add_diversity(
                    beam_seqs,
                    logprobs,
                    time,
                    group_ix,
                    penalty_lambda,
                    beam_ratio
                )

                # Runs one step of beam_search for the current group
                (
                    beam_seqs[group_ix],
                    beam_seq_logprobs[group_ix],
                    beam_logprobs_sums[group_ix],
                    states[group_ix],
                    candidates_group
                ) = __beam_step(
                        logprobs,
                        aug_logprobs,
                        beam_ratio,
                        beam_seqs[group_ix],
                        beam_seq_logprobs[group_ix],
                        beam_logprobs_sums[group_ix],
                        states[group_ix],
                        time - group_ix
                )

                for beam_ix in range(beam_ratio):
                    is_first_end_token = (
                        (
                            beam_seqs[group_ix][:,beam_ix][time-group_ix] == \
                         end_token
                        ) and (
                            torch.eq(
                                beam_seqs[group_ix][:,beam_ix],
                                end_token
                            ).sum() == 1
                        )
                    )

                    final_time_wo_end = (
                        (
                            time == seq_length + group_ix
                        ) and (
                            torch.eq(
                                beam_seqs[group_ix][:,beam_ix],
                                end_token
                            ).sum() == 0
                        )
                    )

                    if is_first_end_token or final_time_wo_end:
#                         print("End token reached!")
#                         print("time: {}, group_ix: {}, beam_index: {}".format(
#                             time, group_ix, beam_ix
#                         ))
                        final_beam = {
                            "seq": beam_seqs[group_ix][:,beam_ix].clone(),
                            "logps": beam_seq_logprobs[group_ix]\
                                [:,beam_ix].clone(),
                            "logp": beam_seq_logprobs[group_ix][:,beam_ix]\
                                .sum(),
                            "aug_logp": beam_logprobs_sums[group_ix][beam_ix]
                        }
                        final_beam["candidate"] = candidates_group[beam_ix]
#                         print([rev_word_map[int(word)] for word in final_beam['seq']])
                        done_beams[group_ix*beam_ratio+beam_ix] = final_beam

                    if is_first_end_token:
                        beam_logprobs_sums[group_ix][beam_ix] = -1000000

                inpt = beam_seqs[group_ix][time - group_ix]
                output = gen_logprobs(inpt, states[group_ix])
#                 print("Type of output: {}".format(type(output)))
#                 for indexofiteminoutput, iteminoutput in enumerate(output):
#                     print("type of output[{}]: {}".format(
#                         indexofiteminoutput, type(iteminoutput)
#                     ))
#                     print("Shape of output[{}]: {}".format(
#                         indexofiteminoutput, iteminoutput.size()
#                     ))
#                 print("Shape of Last element in output: {}".format(output[-1].size()))
                logprobs_list[group_ix] = output[-1].clone()
                temp_state = [
                    output[i].clone().to(device)
                    for i
                    in range(len(output)-1)
                ]
                states[group_ix] = [st.clone().to(device) for st in temp_state]

#     outputs = []
#     for i in range(num_groups):
#         curr_outs = [[] for s in range(beam_ratio)]
#         group = beam_seqs[i]
#         print(group.size())
#         for j in range(group.size()[1]):
#             for k in range(group.size()[0]):
#                 curr_outs[j].append(rev_word_map[int(group[k,j])])
#         outputs.extend(curr_outs)
#     input()
#     for op in outputs:
#         print(op)
#     input()
#     for i in range(num_groups):
#         done_beams[i].sort(key=lambda x:x['aug_logp'])
#         done_beams[i] = done_beams[i][0:beam_ratio]
#     print(done_beams)
#     for beam in done_beams:
#         caption = []
#         for item in beam['seq']:
#             caption.append(rev_word_map[int(item)])
#         print(caption)
    return done_beams


In [9]:
def caption_image_beam_search(
        encoder,
        decoder,
        image_path,
        word_map,
        beam_size,
        num_groups,
        penalty):
    """Reads an image and captions it with beam search.
    param encoder: encoder model
    param decoder: decoder model
    param image_path: path to image
    param word_map: word map
    param beam_size: number of sequences to consider at each decode-step
    return: caption, weights for visualization
    """

    k = beam_size
    beam_ratio = int(k/num_groups)
    vocab_size = len(word_map)

    # Read image and process
    img = imread(image_path)

    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]
        img = np.concatenate([img, img, img], axis=2)
    img = imresize(img, (256, 256))
    img = img.transpose(2, 0, 1)
    img = img / 255.
    img = torch.FloatTensor(img)
    img = img.to(device)
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)

    # Encode
    image = img.unsqueeze(0)  # (1, 3, 256, 256)
    encoder_out = encoder(image)
                # (1, enc_image_size, enc_image_size, encoder_dim)
    enc_image_size = encoder_out.size(1)
    encoder_dim = encoder_out.size(3)

    # Flatten encoding
    encoder_out = encoder_out.view(1, -1, encoder_dim)
    # (1, num_pixels, encoder_dim)

    num_pixels = encoder_out.size(1)

    # We'll treat the problem as having a batch size of k
    encoder_out = encoder_out.expand(
        beam_ratio,
        num_pixels,
        encoder_dim
    )  # (k, num_pixels, encoder_dim)

    # Tensor to store top k previous words at each step;
    #now they're just <start>
    # k_prev_words = torch.LongTensor(
    #     [[word_map['<start>']]]
    # ).to(device)  # (k, 1)

    # init_logprobs = [([None] * beam_ratio)*num_groups]

    h, c = decoder.init_hidden_state(encoder_out)
#     print("h: ", h.shape)
#     print("c: ", c.shape)

    def generate_log_probabilities(inputs, states):
        h_ = states[0]
        c_ = states[1]
        embeddings = decoder.embedding(inputs).squeeze(1)
        awe, alpha = decoder.attention(encoder_out, h_)
#         print("awe before gate*awe: " + str(awe.shape))
        gate = decoder.sigmoid(decoder.f_beta(h_))
        awe = gate * awe
#         print("awe after gate*awe: " + str(awe.shape))
        
#         print("embeddings: "+ str(embeddings.shape))
#         print("concatenated: ",torch.cat([embeddings, awe], dim=1).size())
        h_, c_ = decoder.decode_step(
            torch.cat([embeddings, awe], dim=1),
            (h_, c_)
        )  #tried something
        scores = F.log_softmax(decoder.fc(h_), dim=1)
        return h_, c_, scores

    init_seq = torch.LongTensor([[word_map['<start>']]]*beam_ratio).to(device)
#     print("Sizeof Init_Seq: {}".format(init_seq.size()))
    # init_seq = torch.LongTensor([word_map['<start>']]*beam_ratio).to(device).unsqueeze(0)
    h,c,init_logprobs = generate_log_probabilities(init_seq, [h,c])

    done_beams = diverse_beam_search(
        rnn_state=[h, c],
        init_logprobs=init_logprobs,
        num_beams=k,
        num_groups=num_groups,
        penalty_lambda=penalty,
        seq_length=50,
        end_token=word_map['<end>'],
        gen_logprobs=generate_log_probabilities,
        init_token=word_map['<start>']
    )
    # s is a number less than or equal to k,
    #because sequences are removed from this process once they hit <end>
#     for beam in done_beams:
#         print(type(beam))
#         print(beam)
    seq = [beam['seq'] for beam in done_beams]
    return seq


In [10]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, target):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.target = target

In [11]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""
    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


In [12]:
def convert_sentence_pair(score, first_sentences, second_sentences, max_seq_length, tokenizer):
    features = []
    sum = 0
    for (ex_index, (score, first_sentence, second_sentence)) in enumerate(zip(score, first_sentences, second_sentences)):
        tokens_a = tokenizer.tokenize(first_sentence)
        tokens_b = tokenizer.tokenize(str(second_sentence))
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        if(len(tokens_b) > max_seq_length-3):
            sum +=1
        elif(len(tokens_a) > max_seq_length-3):
            sum +=1
            
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the bert_model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire bert_model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        # if ex_index < 5:
        #     logger.info("*** Example ***")
        #     logger.info("tokens: %s" % " ".join(
        #             [str(x) for x in tokens]))
        #     logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        #     logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
        #     logger.info(
        #             "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))

        features.append(
                InputFeatures(
                    input_ids=input_ids,
                    input_mask=input_mask,
                    segment_ids=segment_ids,
                    target=score/5.0
        ))
    # print(sum)  
    return features

In [15]:
model_path = "./pretrained_model/BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar"
word_map_path = "./pretrained_model/WORDMAP_coco_5_cap_per_img_5_min_word_freq.json"
# image_path = "./animal_khichdi.jpeg"
beam_size = 8 # keep num group == num beams, or prepare for trouble
num_groups = 8 # and make it double
_lambda = 0.5
img_path = "../Jas_specificity/data/images/memorability/"
similarity_model = '../BERT finetune/saved_models/pytorch_finetuned_BERT.bin'

In [16]:
# parser = argparse.ArgumentParser(description='Show, Attend, and Tell - Tutorial - Generate Caption')

# parser.add_argument('--img_path', '-i', help='path to images')
# parser.add_argument('--captioning_model', '-cm', help='path to captioning model')
# parser.add_argument('--word_map', '-wm', help='path to word map JSON')
# parser.add_argument('--beam_size', '-b', default=8, type=int, help='beam size for beam search')
# parser.add_argument('--dont_smooth', dest='smooth', action='store_false', help='do not smooth alpha overlay')
# parser.add_argument('--lambda', default=0.5, type=float, dest="_lambda", help="Diversity parameter for diverse beam search")
# parser.add_argument('--similarity_model', '-sm', help='path to similarity model')

# args = parser.parse_args()
# print("ARGS:\n",args)

checkpoint = torch.load(model_path)
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval()

tokenizer = BertTokenizer.from_pretrained(
    "bert-base-uncased", do_lower_case=True, 
    cache_dir=PYTORCH_PRETRAINED_BERT_CACHE)

bert_model = BertForNextSentencePrediction.from_pretrained(
    "bert-base-uncased",
    cache_dir=PYTORCH_PRETRAINED_BERT_CACHE
)
print(type(bert_model))
bert_model.load_state_dict(torch.load(similarity_model))

bert_model.to(device)
bert_model.eval()

with open(word_map_path, 'r') as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}

imgs = os.listdir(img_path)
mb = progress_bar(imgs)
results_dict = {}

for img_file in mb:
    img = os.path.join(img_path, img_file)
    print('Image name: ', img_file)

    try:
        seqs = caption_image_beam_search(
                encoder,
                decoder,
                img,
                word_map,
                beam_size=beam_size,
                num_groups=num_groups,
                penalty=_lambda
            )
        
        sentences = []

        for seq in seqs:
            word = ''
            words = ""
            for ind in seq:
                word = rev_word_map[int(ind)]
                if word == '<end>':
                    break        
                words+= (word + " ")
            print(words)
            sentences.append(words)
        
        results_dict[img_file] = dict()
        results_dict[img_file]['Captions'] = sentences 
            
        print()
        print()
        print()

        sentence_pairs = list(itertools.combinations(sentences, 2))

        df_infer = pd.DataFrame(sentence_pairs, columns=['first_sentence', 'second_sentence'])
        df_infer['score'] = 0

        correct_pairs = convert_sentence_pair(
            df_infer.score.tolist(), 
            df_infer.first_sentence.tolist(), 
            df_infer.second_sentence.tolist(), 
            max_seq_length=128, 
            tokenizer=tokenizer
        )

        BATCH_SIZE = 1
        
        logger.info("***** Comparing Sentence Pairs *****")
        all_input_ids = torch.tensor([f.input_ids for f in correct_pairs], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in correct_pairs], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in correct_pairs], dtype=torch.long)
        infer_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
        # Run prediction for full data
        infer_sampler = SequentialSampler(infer_data)
        infer_dataloader = DataLoader(infer_data, sampler=infer_sampler, batch_size=BATCH_SIZE)

        logger.info("  Num pairs = %d", len(correct_pairs))
        logger.info("  Batch size = %d", BATCH_SIZE)
        
        res = []

        mb = progress_bar(infer_dataloader)

        for input_ids, input_mask, segment_ids in mb:
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)
            with torch.no_grad():
                res.append(nn.functional.softmax(
                    bert_model(input_ids, segment_ids, input_mask), dim=1
                )[:, 0].detach().cpu().numpy())

        mean_similarity_score = np.mean(res)
        max_similarity_score = np.max(res)
        min_similarity_score = np.min(res)
        
        logger.info("computing similarity")
        print("Similarity Score: {}".format(mean_similarity_score))
        results_dict[img_file]['meanScore'] = mean_similarity_score.item()
        results_dict[img_file]['maxScore'] = max_similarity_score.item()
        results_dict[img_file]['minScore'] = min_similarity_score.item()
        
    except Exception as inst:
        print(type(inst))    # the exception instance
        print(inst.args)     # arguments stored in .args
        print(inst)
with open("{}/../results.json".format(img_path), 'w') as outfile:
    json.dump(results_dict, outfile)

03/29/2019 10:26:41 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ~/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/29/2019 10:26:41 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ~/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/29/2019 10:26:41 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ~/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpyh8hxw2k
03/29/2019 10:26:45 - INFO - pytorch_pretrai

<class 'pytorch_pretrained_bert.modeling.BertForNextSentencePrediction'>


Image name:  img_313.jpg


03/29/2019 10:26:49 - INFO - bert -   ***** Comparing Sentence Pairs *****
03/29/2019 10:26:49 - INFO - bert -     Num pairs = 28
03/29/2019 10:26:49 - INFO - bert -     Batch size = 1


a small wooden bench sitting in front of a house 
a wooden bench sitting in front of a house 
a small house with a wooden bench and a bench 
a house with a wooden bench and a bench 
a wooden bench sitting in front of a small building 
an old train station with a train on the tracks 
a small building with a bench and a wooden bench in front of it 
the train is going through the yard by the house 





...

Similarity Score: 0.7514539957046509
Image name:  img_365.jpg


03/29/2019 10:42:40 - INFO - bert -   ***** Comparing Sentence Pairs *****
03/29/2019 10:42:40 - INFO - bert -     Num pairs = 28
03/29/2019 10:42:40 - INFO - bert -     Batch size = 1


a view of a person on a hill with a kite in the sky 
a person is flying a kite on a dirt road 
a view of a road with a kite in the distance 
a person on a road with a kite flying in the air 
a view of a field with birds flying over it 
a road with people flying kites in the sky 
the view of a kite flying over a dirt road 
a group of people flying kites in a field 





03/29/2019 10:42:40 - INFO - bert -   computing similarity


Similarity Score: 0.6989150047302246


In [ ]:
# log_file.close()
# sys.stdout = old_stdout

# Load model
checkpoint = torch.load(args.captioning_model)
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval()

# Load word map (word2ix)
with open(args.word_map, 'r') as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

imgs = os.listdir(args.img_path)

for img_file in imgs:
    img = os.path.join(args.img_path, img_file)

    # Encode, decode with attention and beam search
    try:
        seqs, alphas = caption_image_beam_search(encoder, decoder, img, word_map, args.beam_size)
        alphas = torch.FloatTensor(alphas)

        sentences = []

        for seq in seqs:
            words = [rev_word_map[ind] for ind in seq]
            sentences.append(" ".join(words))

        print("Sentences:")
        for sentence in sentences:
            print(sentence)

        print()
        print()
        print()

        sentence_pairs = list(itertools.combinations(sentences, 2))


        df_infer = pd.DataFrame(sentence_pairs, columns=['first_sentence', 'second_sentence'])
        df_infer['score'] = 0

        tokenizer = BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True, 
            cache_dir=PYTORCH_PRETRAINED_BERT_CACHE)

        bert_model = BertForNextSentencePrediction.from_pretrained(
            "bert-base-uncased",
            cache_dir=PYTORCH_PRETRAINED_BERT_CACHE
        )
        print(type(bert_model))
        bert_model.load_state_dict(torch.load(args.similarity_model))

        bert_model.to(device)


        correct_pairs = convert_sentence_pair(df_infer.score.tolist(), df_infer.first_sentence.tolist(), df_infer.second_sentence.tolist(), max_seq_length=128, tokenizer=tokenizer)


        BATCH_SIZE = 1
        
        logger.info("***** Comparing Sentence Pairs *****")
        all_input_ids = torch.tensor([f.input_ids for f in correct_pairs], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in correct_pairs], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in correct_pairs], dtype=torch.long)
        infer_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
        # Run prediction for full data
        infer_sampler = SequentialSampler(infer_data)
        infer_dataloader = DataLoader(infer_data, sampler=infer_sampler, batch_size=BATCH_SIZE)

        logger.info("  Num pairs = %d", len(correct_pairs))
        logger.info("  Batch size = %d", BATCH_SIZE)

        bert_model.eval()

        res = []

        mb = infer_dataloader

        for input_ids, input_mask, segment_ids in tqdm(mb):
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)
            with torch.no_grad():
                res.append(nn.functional.softmax(
                    bert_model(input_ids, segment_ids, input_mask), dim=1
                )[:, 0].detach().cpu().numpy())

        similarity_score = np.mean(res)

        print("Similarity Score: {}".format(similarity_score))
    except:
        pass


log_file.close()

In [ ]:
results_dict["000000010707.jpg"]